# Multilevel Monte Carlo Path Simulation

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys

# Add the path to the mlmc module to the python path
sys.path.append("../src")
from mlmc import MLMC, MonteCarloEstimator
from model import Model, BlackScholes, Heston
from contract import Contract, EuropeanCall, AsianCall, Lookback, Digital

In [2]:
# Models
interest_rate = 0.05
initial_value = np.array([1.0, 0.04])
sigma = 0.2
lbd, xi, rho = 5.0, 0.25, -0.5

bs = BlackScholes(interest_rate, np.array(initial_value[0]), sigma)
heston = Heston(interest_rate, initial_value, sigma, lbd, xi, rho)

# Contracts
maturity = 1.0
strike = 1.0

ue_call = EuropeanCall(maturity, strike)
asian_call = AsianCall(maturity, strike)
lookback = Lookback(maturity, sigma)
digital = Digital(maturity)

In [3]:
def create_paper_plots(mc: MonteCarloEstimator, model: Model, contract: Contract, target_errors: np.ndarray) -> None:
    m = mc.m
    sample_count, level = 10**6, 5

    print("Computations for estimation of mean and variance...")
    estimator = mc.computations_for_plots(model, contract, sample_count, level)

    log_means = np.log(np.cumsum(estimator["means"])) / np.log(m)
    log_means_diff = np.log(estimator["means"][1:]) / np.log(m)
    log_vars = np.log(np.cumsum(estimator["vars"])) / np.log(m)
    log_vars_diff = np.log(estimator["vars"][1:]) / np.log(m)
    log_means_richardson = np.log(estimator["means_richardson"]) / np.log(m)

    optimals_samples = []
    optimals_samples_richardson = []
    computationnal_costs = []
    computationnal_costs_richardson = []
    computationnal_costs_std = []
    computationnal_costs_richardson_std = []
    for target_error in target_errors:
        print("Computing mlmc estimator without richardson extrapolation")
        estimator, samples = mc.compute_multilevel_estimator(model, contract, target_error, False)
        optimals_samples.append(samples)

        print("Computing mlmc estimator with richardson extrapolation")
        estimator, samples_richardson = mc.compute_multilevel_estimator(model, contract, target_error, True)
        optimals_samples_richardson.append(samples_richardson)

        costs = [samples[0]] + [samples[i] * (m**i + m ** (i - 1)) for i in range(1, len(samples))]
        computationnal_costs.append(np.sum(costs))
        costs_richardson = [samples_richardson[0]] + [
            samples_richardson[i] * (m**i + m ** (i - 1)) for i in range(1, len(samples_richardson))
        ]
        computationnal_costs_richardson.append(np.sum(costs_richardson))

        print("Computing standard estimator without richardson extrapolation")
        estimator, samples_std = mc.compute_standard_estimator(model, contract, target_error, False)
        computationnal_costs_std.append(np.sum([s * m**i for i, s in enumerate(samples_std)]))

        # print("Computing standard estimator with richardson extrapolation")
        # estimator, samples_std_richardson = mc.compute_standard_estimator(model, contract, target_error, True)
        # computationnal_costs_richardson_std.append(np.sum([s * m**i for i, s in enumerate(samples_std_richardson)]))

    epsilon_cost = np.array(computationnal_costs) * target_errors**2
    epsilon_cost_richardson = np.array(computationnal_costs_richardson) * target_errors**2
    epsilon_cost_std = np.array(computationnal_costs_std) * target_errors**2
    # epsilon_cost_richardson_std = np.array(computationnal_costs_richardson_std) * target_errors**2

    fig, axs = plt.subplots(2, 2, figsize=(10, 8), dpi=120)
    l1 = np.arange(0, level)
    l2 = np.arange(1, level)
    l3 = np.arange(2, level)

    # Top left
    axs[0, 0].plot(l1, log_vars, "-", marker="x", c="black", label="$P_l$")
    axs[0, 0].plot(l2, log_vars_diff, "--", marker="x", c="black", label="$P_l-P_{l-1}$")
    axs[0, 0].set_ylabel("$log_M$ variance")
    axs[0, 0].set_xlabel("l")
    axs[0, 0].legend()

    # Top right
    axs[0, 1].plot(l1, log_means, "-", marker="x", c="black", label="$P_l$")
    axs[0, 1].plot(l2, log_means_diff, "--", marker="x", c="black", label="$P_l-P_{l-1}$")
    axs[0, 1].plot(l3, log_means_richardson, "-.", marker="x", c="black", label="$Y_l-Y_{l-1}/M$")
    axs[0, 1].set_xlabel("$l$")
    axs[0, 1].set_ylabel("$log_M |mean|$")
    axs[0, 1].legend()

    # Bottom left
    markers = ["x", "o", "v", "s", "*"]
    for i, optimal_samples in enumerate(optimals_samples):
        n = len(optimal_samples)
        l = np.arange(start=0, stop=n, step=1)
        axs[1, 0].plot(l, optimal_samples, ":", marker=markers[i], c="black", label=rf"$\epsilon = {target_error[i]}$")
        axs[1, 0].set_yscale("log")

    for i, optimal_samples in enumerate(optimals_samples_richardson):
        n = len(optimal_samples)
        l = np.arange(start=0, stop=n, step=1)
        axs[1, 0].plot(l, optimal_samples, "-.", marker=markers[i], c="black")
        axs[1, 0].set_yscale("log")

    axs[1, 0].set_xlabel("$l$")
    axs[1, 0].set_ylabel("$N_l$")
    axs[1, 0].legend()

    # Bottom right
    axs[1, 1].plot(target_errors, epsilon_cost, "--", c="black", marker="x", label="MLMC")
    axs[1, 1].plot(target_errors, epsilon_cost_richardson, "-.", c="black", marker="x", label="MLMC ext")
    axs[1, 1].plot(target_errors, epsilon_cost_std, ":", c="black", marker="x", label="std MC")
    # axs[1, 1].plot(
    #     target_errors, epsilon_cost_richardson_std, color="black", linestyle="-", marker="x", label="std MC ext"
    # )

    axs[1, 1].set_xscale("log")
    axs[1, 1].set_yscale("log")
    axs[1, 1].set_ylabel(r"$\epsilon^2$cost")
    axs[1, 1].set_xlabel(r"$\epsilon$")
    axs[1, 1].legend()

    return fig, axs

In [4]:
# MLMC parameters
max_level = 4
default_sample_count = 10_000
m = 4

# For reproducibility
rng = np.random.default_rng(seed=42)

mc = MonteCarloEstimator(max_level, m, default_sample_count, rng)

In [5]:
targets_errors = np.array(
    [
        [0.001, 0.0005, 0.0002, 0.0001, 0.00005],
        [0.001, 0.0005, 0.0002, 0.0001, 0.00005],
        [0.002, 0.001, 0.0005, 0.0002, 0.0001],
        [0.005, 0.002, 0.001, 0.0005, 0.0002],
        [0.005, 0.0005, 0.0002, 0.0001, 0.00005],
    ]
)
models = [bs, bs, bs, bs, heston]
contracts = [ue_call, asian_call, lookback, digital, ue_call]

params = zip(targets_errors, models, contracts)
plots = []

for target_errors, model, contract in params:
    # plots.append(create_paper_plots(mc, model, contract, target_errors))
    fig, _ = create_paper_plots(mc, model, contract, target_errors)
    fig.show()
    plt.show()

# for fig, _ in plots:
#     fig.show()

plt.show()

Computations for estimation of mean and variance...


KeyboardInterrupt: 